<a href="https://colab.research.google.com/github/leonardo-muniz/IMDB-Sentiment-Analysis/blob/master/imdb_sentiment_analysis_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

In [2]:
from datasets import load_dataset

In [3]:
def load_and_inspect_dataset():
  """Carrega o dataset IMDB e exibe informações básicas."""
  print("Iniciando o carregamento do dataset IMDB...")
  try:
    # Carrega o dataset do Hugging Face
    dataset = load_dataset("imdb")

    train_data = dataset["train"]
    test_data = dataset["test"]

    print(f"Reviews no conjunto de treino: {len(train_data)}")
    print(f"Reviews no conjunto de teste: {len(test_data)}")

    print("\nCarregamento e inspeção básicos concluídos. Dataset pronto para pré-processamento!")

    return train_data, test_data

  except Exception as e:
    print(f"\nOcorreu um erro ao carregar o dataset: {e}")
    return None, None


In [4]:
train_data, test_data = load_and_inspect_dataset()

Iniciando o carregamento do dataset IMDB...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Reviews no conjunto de treino: 25000
Reviews no conjunto de teste: 25000

Carregamento e inspeção básicos concluídos. Dataset pronto para pré-processamento!


In [5]:
from transformers import AutoTokenizer

# Carregar o tokenizer para um modelo popular de classificação de texto
# 'distilbert-base-uncased' é uma boa escolha para começar: leve e eficiente.
# 'uncased' significa que o texto será convertido para minúsculas automaticamente.
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

print("Tokenizer carregado com sucesso!")
print(f"Exemplo de como o tokenizer funciona para uma frase simples: {tokenizer('Isso é um teste de tokenização.')}")

Tokenizer carregado com sucesso!
Exemplo de como o tokenizer funciona para uma frase simples: {'input_ids': [101, 26354, 2080, 1041, 8529, 3231, 2063, 2139, 19204, 21335, 20808, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [6]:
def preprocess_function(examples):
    # O tokenizer lida com a tokenização, conversão para IDs, e adição de attention_mask.
    # truncation=True garante que as sequências mais longas que o limite do modelo sejam cortadas.
    # max_length pode ser especificado se você quiser um comprimento máximo diferente do padrão do modelo.
    return tokenizer(examples["text"], truncation=True)

# Aplicar a função de pré-processamento aos conjuntos de treino e teste
# batched=True processa os exemplos em lotes, o que é mais eficiente.
# remove_columns=['text'] remove a coluna de texto original, pois não precisaremos dela mais.
# Isso ajuda a economizar memória e focar nas colunas processadas.
tokenized_train_data = train_data.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_test_data = test_data.map(preprocess_function, batched=True, remove_columns=["text"])

print("\nDataset de treino tokenizado:")
print(tokenized_train_data)
print("\nDataset de teste tokenizado:")
print(tokenized_test_data)

# Você pode inspecionar um exemplo para ver as novas colunas
print("\nExemplo de dado tokenizado (primeiro item do treino):")
print(tokenized_train_data[0])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]


Dataset de treino tokenizado:
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

Dataset de teste tokenizado:
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 25000
})

Exemplo de dado tokenizado (primeiro item do treino):
{'label': 0, 'input_ids': [101, 1045, 12524, 1045, 2572, 8025, 1011, 3756, 2013, 2026, 2678, 3573, 2138, 1997, 2035, 1996, 6704, 2008, 5129, 2009, 2043, 2009, 2001, 2034, 2207, 1999, 3476, 1012, 1045, 2036, 2657, 2008, 2012, 2034, 2009, 2001, 8243, 2011, 1057, 1012, 1055, 1012, 8205, 2065, 2009, 2412, 2699, 2000, 4607, 2023, 2406, 1010, 3568, 2108, 1037, 5470, 1997, 3152, 2641, 1000, 6801, 1000, 1045, 2428, 2018, 2000, 2156, 2023, 2005, 2870, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 5436, 2003, 8857, 2105, 1037, 2402, 4467, 3689, 3076, 2315, 14229, 2040, 4122, 2000, 4553, 2673, 2016, 2064, 2055, 2166, 1012, 1999, 3327, 2016, 4122, 2000, 3579, 2014, 3086, 2015, 2000, 2437, 2070, 

In [7]:
# First, ensure 'evaluate' is installed if you haven't already:
!pip install evaluate

In [8]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate # Corrected import

# 1. Mapear os rótulos (labels)
# O dataset IMDB já tem labels 0 e 1. Para classificação, 0 e 1 são bons.
# Podemos renomear a coluna 'label' para 'labels' que é o padrão esperado pelo Trainer.
tokenized_train_data = tokenized_train_data.rename_column("label", "labels")
tokenized_test_data = tokenized_test_data.rename_column("label", "labels")

# Definir as colunas que serão usadas no treinamento
tokenized_train_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test_data.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


# 2. Definir um DataCollator
# Este data collator irá dinamicamente padar os batches para o comprimento máximo em cada batch.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Corrected typo DataatorCollatorWithPadding to DataCollatorWithPadding
print("\nDataCollator com padding definido.")

# 3. Carregar o Modelo
# Vamos usar o mesmo modelo que o tokenizer (distilbert-base-uncased) para classificação de sequência.
# num_labels=2 porque temos duas classes: positivo e negativo.
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
print("\nModelo DistilBERT para classificação de sequência carregado.")

# 4. Definir métricas de avaliação
# Precisamos de uma função para calcular métricas durante o treinamento.
# Acurácia é uma boa métrica para classificação.
metric = evaluate.load("accuracy") # Corrected: use evaluate.load

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

print("Função de métricas de avaliação definida.")


DataCollator com padding definido.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Modelo DistilBERT para classificação de sequência carregado.
Função de métricas de avaliação definida.


In [ ]:
# 5. Configurar os argumentos de treinamento
training_args = TrainingArguments(
    output_dir="./results",               # Diretório para salvar os resultados do treinamento e checkpoints
    num_train_epochs=3,                   # Número de épocas de treinamento (3 é um bom ponto de partida)
    per_device_train_batch_size=16,       # Tamanho do batch para treinamento em cada GPU/CPU
    per_device_eval_batch_size=16,        # Tamanho do batch para avaliação em cada GPU/CPU
    warmup_steps=500,                     # Número de passos para aquecimento da taxa de aprendizado
    weight_decay=0.01,                    # Taxa de decaimento de peso para regularização
    logging_dir="./logs",                 # Diretório para salvar os logs do TensorBoard
    logging_steps=500,                    # Frequência de log dos passos de treinamento
    eval_strategy="epoch",                # CORRIGIDO: Avaliar no final de cada época (era evaluation_strategy)
    save_strategy="epoch",                # Salvar checkpoint no final de cada época
    load_best_model_at_end=True,          # Carregar o melhor modelo com base na métrica de avaliação
    metric_for_best_model="accuracy",     # Métrica para determinar o melhor modelo
    push_to_hub=False,                    # Não enviar para o Hugging Face Hub neste momento
)

# 6. Criar e iniciar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_test_data,
    tokenizer=tokenizer, # É bom passar o tokenizer para que o Trainer saiba como lidar com o padding, mesmo com o data_collator
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\nConfigurações do Trainer concluídas. Iniciando o treinamento...")

# Iniciar o treinamento
trainer.train()

print("\nTreinamento concluído!")

# Você também pode avaliar o modelo no conjunto de teste após o treinamento
print("\nAvaliação final no conjunto de teste:")
results = trainer.evaluate()
print(results)

# Adicionado para salvar o modelo e o tokenizer após o treinamento e avaliação
final_model_save_path = "./my_imdb_sentiment_model" # Define o caminho para salvar
trainer.save_model(final_model_save_path) # Salva o modelo (já o melhor carregado pelo load_best_model_at_end)
tokenizer.save_pretrained(final_model_save_path) # Salva o tokenizer junto com o modelo

print(f"\nModelo e Tokenizer salvos em: {final_model_save_path}")

/tmp/ipython-input-9-662013283.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Configurações do Trainer concluídas. Iniciando o treinamento...


Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# 1. Definir o caminho onde seu modelo e tokenizer foram salvos
# Use o mesmo caminho que você definiu em final_model_save_path no código anterior
model_load_path = "./my_imdb_sentiment_model"

# 2. Carregar o tokenizer e o modelo do disco
# Isso é crucial para não precisar retreinar toda vez
print(f"Carregando modelo e tokenizer de: {model_load_path}...")
loaded_tokenizer = AutoTokenizer.from_pretrained(model_load_path)
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_load_path)
print("Modelo e Tokenizer carregados com sucesso!")

# 3. Criar uma pipeline de análise de sentimento
# A pipeline abstrai a tokenização e o pós-processamento, tornando a inferência simples.
sentiment_pipeline = pipeline(
    "sentiment-analysis", # Tipo de tarefa
    model=loaded_model,   # Seu modelo carregado
    tokenizer=loaded_tokenizer # Seu tokenizer carregado
)

# 4. Dar uma frase (ou uma lista de frases) para o modelo classificar
print("\nRealizando previsões...")

frase1 = "Este filme é uma obra-prima, absolutamente imperdível!"
frase2 = "Que perda de tempo! O pior filme que vi este ano."
frase3 = "É um filme mediano, com alguns momentos bons e outros nem tanto."
frase4 = "As atuações são excelentes, mas o roteiro deixa a desejar."

# Classificar uma única frase
results1 = sentiment_pipeline(frase1)
print(f"\nFrase: '{frase1}'")
# A saída será algo como [{'label': 'LABEL_1', 'score': 0.99...}]
# Lembre-se que LABEL_0 é negativo e LABEL_1 é positivo no seu dataset
label1 = "Positivo" if results1[0]['label'] == 'LABEL_1' else "Negativo"
print(f"Sentimento: {label1} (Confiança: {results1[0]['score']:.4f})")

# Classificar múltiplas frases de uma vez (mais eficiente)
frases_para_classificar = [frase1, frase2, frase3, frase4]
results_batch = sentiment_pipeline(frases_para_classificar)

print("\nClassificação de Múltiplas Frases:")
for i, result in enumerate(results_batch):
    current_label = "Positivo" if result['label'] == 'LABEL_1' else "Negativo"
    print(f"Frase: '{frases_para_classificar[i]}'")
    print(f"Sentimento: {current_label} (Confiança: {result['score']:.4f})\n")